In [ ]:
class IllegalCellValue(Exception):
    pass

    def __init__(self, msg=None):
        Exception.__init__(self)
        if msg is not None:
            print(msg)

class Cell(object):
    VALID_CELL_VALUES = {1, 2, 3, 4, 5, 6, 7, 8, 9}
    
    def __init__(self, r, c):
        self._r = r
        self._c = c
        self._id = "cell_%s" % (tuple([self._r, self._c]),)
        self._parents = {}
        self._value = None
        
    def add_parent(self, parent, parent_type):
        self._parents[parent_type] = parent
        
    def set_value(self, v):
        if v not in self.VALID_CELL_VALUES:
            raise IllegalCellValue
            
        if self._value is not None:
            raise IllegalCellValue
            
        # test every parent, then do it for real
        for testing in [True, False]:
            for k, p in self._parents.items():
                p.add_value(v, testing=testing)
        
        self._value = v
        
    def clear_value(self):
        if self._value is None:
            raise IllegalCellValue(msg="%s does not contain a value" % (self._id))
        
        for k, p in self._parents.items():
            p.delete_value(self._value)

        self._value = None
            
# class Threes(object):
#     pass
    
class Nines(object):

    def __init__(self, t, index):
        self._type = t
        self._index = index
        self._id = "%s_%s" % (self._type, self._index)
        self._cells = {}
        # print("__init__ id: %s" % self._id)
        self._child_values = set()
        
    def add_cell(self, cell):
        self._cells[cell._id] = cell
        cell.add_parent(self, self._type)

    def add_value(self, v, testing=False):
        if v not in Cell.VALID_CELL_VALUES:
            raise IllegalCellValue
        
        if v in self._child_values:
            raise IllegalCellValue(msg="%s already has cell with value %s" % (self._id, v))
        if not testing:
            self._child_values.add(v)
    
    def clear_value(self, v, testing=False):
        if v not in Cell.VALID_CELL_VALUES:
            raise IllegalCellValue
        
        if v not in self._child_values:
            raise IllegalCellValue(msg="%s does not contain a cell with value %s" % (self._id, v))
        
        if not testing:
            try:
                self._child_values.remove(v)
            except KeyError:
                raise IllegalCellValue
        
class Board(object):
    NUM_ROWS = 9
    NUM_COLS = NUM_ROWS
    ROW_NUMS = range(NUM_ROWS)
    COL_NUMS = range(NUM_COLS)
    SQUARE_COORDINATES = [] 
    
    def __init__(self):
        self.__class__.SQUARE_COORDINATES = [(r,c) for r in range(0, self.__class__.NUM_ROWS, 3) for c in range(0, self.__class__.NUM_COLS, 3)] 

        
        self._all_cells = {(r,c):Cell(r,c) for r in self.__class__.ROW_NUMS for c in self.__class__.COL_NUMS}
        
        self._all_rows = {r:Nines("row", r) for r in self.__class__.ROW_NUMS}
        self._all_cols = {c:Nines("col", c) for c in self.__class__.COL_NUMS}
        self._all_squares = {r_c:Nines("square", r_c) for r_c in self.__class__.SQUARE_COORDINATES}
        
        self._all_nines = dict()
        self._all_nines.update({v._id:v for v in self._all_rows.values()})
        self._all_nines.update({v._id:v for v in self._all_cols.values()})
        self._all_nines.update({v._id:v for v in self._all_squares.values()})
        
        for cell in self._all_cells.values():
            r, c = (cell._r, cell._c)
            row = self._all_rows[r].add_cell(cell)
            col = self._all_cols[c].add_cell(cell)
            sr = (r // 3) * 3
            sc = (c // 3) * 3
            square = self._all_squares[(sr, sc)].add_cell(cell)
            
        pass

    def get_cell(self, r, c):
        return self._all_cells[(r,c)]
    
    def debug_init(self):
        print("debug - %s" % (self.__class__ )) 
        
        print("debug - parents of cell(4,4): %s" % ([p._id for p in self.get_cell(4,4)._parents.values()]))
        print("debug - parents of cell(4,5): %s" % ([p._id for p in self.get_cell(4,5)._parents.values()]))
        
        # print("debug - nines keys: %s" % self._all_nines.keys())
        # print("debug - squares: %s" % self._all_squares)
        # print("debug - square coordinates: %s" % self.SQUARE_COORDINATES)
        # print("All rows: %s" % self._all_rows)
        # print("All columns: %s" % self._all_cols)
        
    def dump_all_nines(self):
        all_nines = list()
        all_nines += self._all_rows.values()
        all_nines += self._all_cols.values()
        all_nines += self._all_squares.values()
        
        for p in all_nines:
            print("%s has values %s" % (p._id, p._child_values))


In [ ]:
print("testing1()")

b = Board()
b.debug_init()

c44 = b.get_cell(4, 4)
c45 = b.get_cell(4, 5)

# c44.clear_value()  # should fail
c44.set_value(5)

# c45.set_value(5)  # should fail
c45.set_value(4)

b.dump_all_nines()


In [ ]:
if __name__ == "__main__":
    testing = True
    
    if not testing:
        main()
    else:
        testing1()

In [ ]:
def testing9():
    print("testing9()")
    
    sq = Nines([1, 2, 3, 4])
    row = Nines([3, 5, 7, 8, 9])
    col = Nines([4, 2, 1, 8])

    my_guesses = Nines.Cell.VALID_CELL_VALUES.difference(sq._vals, row._vals, col._vals)

    print("all values: %s" % Nines.Cell.VALID_CELL_VALUES)
    print("my guesses: %s" % my_guesses)
    print(len(my_guesses))
